In [0]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import math
import matplotlib.cm as cm
from termcolor import colored
'''
1. Data
 - Config
 - Load Data
'''

# Config
minimum_value=1e-10
over_lambda=1e-5
right_lambda=1e-1
under_lambda=1


learning_rate=0.08
opt_threshold=0.000002748
over_theta=np.random.normal(0., 1, (1, 100))
right_theta=np.random.normal(0., 1, (1, 100))
under_theta=np.random.normal(0., 1, (1, 100))
#over_theta=np.expand_dims(np.ones(100), axis=0)
#right_theta=np.expand_dims(np.ones(100), axis=0)
#under_theta=np.expand_dims(np.ones(100), axis=0)

#t0, t1, t2, t3, t4, t5, t6=1., 1., 1., 1., 1., 1., 1.
#t7, t8, t9, t10, t11, t12, t13=1., 1., 1., 1., 1., 1., 1.
#theta=np.array([[t0, t1, t2, t3, t4, t5, t6,
#                 t7, t8, t9, t10, t11, t12, t13]])


# Load data
path = "drive/My Drive/Classroom/Machine Learning (2) 2020-1\
/class-MachineLearning/assignment07/"
train=[]
test=[]

data = np.genfromtxt(path+"data-nonlinear.txt", delimiter=',')

pointX  = data[:, 0]
pointY  = data[:, 1]

label   = data[:, 2]
m=len(pointX)
pointX0 = pointX[label == 0]
pointY0 = pointY[label == 0]

pointX1 = pointX[label == 1]
pointY1 = pointY[label == 1]


for i in range(10) :
    for j in range(10) :
        if i==j==0 :
            input_data=(pointX**i * (pointY**j)).reshape(-1, 1)
            continue
        t=(pointX**i) * (pointY**j)
        input_data=np.hstack((input_data, t.reshape(-1, 1)))


In [0]:
'''
2. Hypothesis
 - Define hypothesis and sigmoid hypothesis.

3. Objective function
 - Define objective function.

4. Gradient Descent
 - Define derivations.
 - Update thetas
'''

# Define hypothesis
def hypothesis() :
    return (input_data.dot(over_theta.T).flatten(), \
            input_data.dot(right_theta.T).flatten(), \
            input_data.dot(under_theta.T).flatten())

def sigmoid_h() :
    return (1/(1+np.exp(-over_z)), 1/(1+np.exp(-right_z)), 1/(1+np.exp(-under_z)))
    
over_z, right_z, under_z=hypothesis()
over_h, right_h, under_h=sigmoid_h()

# Define objective function

def objective_function() :
    positive_overz=np.array(list(map(lambda x : max(x, 0), over_z)))
    positive_rightz=np.array(list(map(lambda x : max(x, 0), right_z)))
    positive_underz=np.array(list(map(lambda x : max(x, 0), under_z)))
    over_J = np.sum(positive_overz - over_z*label + np.log(1+np.exp(-np.abs(over_z)))) / m + over_lambda/2 * (np.sum(over_theta**2))
    right_J = np.sum(positive_rightz - right_z*label + np.log(1+np.exp(-np.abs(right_z)))) / m + right_lambda/2 * (np.sum(right_theta**2))
    under_J = np.sum(positive_underz - under_z*label + np.log(1+np.exp(-np.abs(under_z)))) / m + under_lambda/2 * (np.sum(under_theta**2))
    #over_J = np.sum(-label*np.log(over_h) - (1-label)*np.log(1-over_h)) / m + over_lambda/2 * (np.sum(over_theta**2))
    #right_J = np.sum(-label*np.log(right_h) - (1-label)*np.log(1-right_h)) / m + right_lambda/2 * (np.sum(right_theta**2))
    #under_J = np.sum(-label*np.log(under_h) - (1-label)*np.log(1-under_h)) / m + under_lambda/2 * (np.sum(under_theta**2))
    return over_J, right_J, under_J

# Define derivations
def derivate() :    
    global over_dev, right_dev, under_dev
    for i in range(len(over_theta[0])) :
        over_dev[i] = np.sum((over_h-label) * input_data[:,i]) / m + (over_lambda * over_theta[0][i])
    for i in range(len(over_theta[0])) :
        right_dev[i] = np.sum((right_h-label) * input_data[:,i]) / m + (right_lambda * right_theta[0][i])
    for i in range(len(over_theta[0])) :
        under_dev[i] = np.sum((under_h-label) * input_data[:,i]) / m + (under_lambda * under_theta[0][i])

# Update thetas
def gradient_descent() :
    global over_theta, right_theta, under_theta
    for i in range(len(over_theta[0])) :
        over_theta[0][i] = over_theta[0][i] - learning_rate * over_dev[i]
    for i in range(len(over_theta[0])) :
        right_theta[0][i] = right_theta[0][i] - learning_rate * right_dev[i]
    for i in range(len(over_theta[0])) :
        under_theta[0][i] = under_theta[0][i] - learning_rate * under_dev[i]
    
def accuracy() :
    over_p=np.array(over_h>=0.5, dtype=np.float64)
    over_c=np.array(label==over_p, dtype=np.float64)
    right_p=np.array(right_h>=0.5, dtype=np.float64)
    right_c=np.array(label==right_p, dtype=np.float64)
    under_p=np.array(under_h>=0.5, dtype=np.float64)
    under_c=np.array(label==under_p, dtype=np.float64)
    return (np.sum(over_c)/m, np.sum(right_c)/m, np.sum(under_c)/m)

over_J_list=[]
right_J_list=[]
under_J_list=[]
over_accuracy_list=[]
right_accuracy_list=[]
under_accuracy_list=[]
over_h_list=[]
right_h_list=[]
under_h_list=[]

over_dev=[0 for i in range(len(over_theta[0]))]
right_dev=[0 for i in range(len(over_theta[0]))]
under_dev=[0 for i in range(len(over_theta[0]))]

# Train
count=0
for i in range(3000) :
    
    over_h_list.append(over_h)
    right_h_list.append(right_h)
    under_h_list.append(under_h)
    #if i>=10000 and accuracy() > accuracy_list[-1] :
    #    learning_rate=0.00001
    over_a, right_a, under_a = accuracy()
    over_accuracy_list.append(over_a)
    right_accuracy_list.append(right_a)
    under_accuracy_list.append(under_a)
    count+=1
    overj, rightj, underj=objective_function()
    over_J_list.append(overj)
    right_J_list.append(rightj)
    under_J_list.append(underj)
    
    derivate()
    
    gradient_descent()
    over_z, right_z, under_z=hypothesis()
    over_h, right_h, under_h=sigmoid_h()


X = np.array([1. for i in range(len(over_theta[0]))])
def H(x, y) :
    global X
    #X=[]
    for i in range(10) :
        for j in range(10) :
            X[10*i + j]=(x**i) * (y**j)
    
    over_H=X.dot(over_theta.T)
    right_H=X.dot(right_theta.T)
    under_H=X.dot(under_theta.T)
    over_grid_H.append(1/(1+np.exp(-over_H)))
    right_grid_H.append(1/(1+np.exp(-right_H)))
    under_grid_H.append(1/(1+np.exp(-under_H)))

'''
For plot contour
'''
gridx=np.linspace(start=-1, stop=1, num=101)
gridy=np.linspace(start=-1, stop=1, num=101)
# Construct meshgrid between x and y
grid_x, grid_y=np.meshgrid(gridx, gridy)      

over_grid_H=[]
right_grid_H=[]
under_grid_H=[]


for (x, y) in zip(np.ravel(grid_x), np.ravel(grid_y)) :
    H(x, y)

over_grid_H=np.array(over_grid_H).reshape(grid_x.shape)
right_grid_H=np.array(right_grid_H).reshape(grid_x.shape)
under_grid_H=np.array(under_grid_H).reshape(grid_x.shape)


# 1. Plot the training data

---
\
     
* data-nonlinear.txt 파일을 동일 디렉터리에 위치시키고 데이터를 읽었습니다.     
 
* 읽은 데이터들을 visualize할때 label이 0인 data는 blue, 1인 data는 red로 시각화했습니다. 

In [0]:
'''
Visualize Data
'''
plt.figure()
plt.scatter(pointX0, pointY0, c='b')
plt.scatter(pointX1, pointY1, c='r')
plt.tight_layout()
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

# 2. Plot the training error

---
\
 - 각각의 lambda에 대해서 loss가 어떻게 변하는지 시각화했습니다.


In [0]:
'''
Visualize Loss
'''
plt.figure(figsize=(10, 7))
plt.plot(over_J_list, color='red', label='over-fitting')
plt.plot(right_J_list, color='green', label='just-right')
plt.plot(under_J_list, color='blue', label='under-fitting')
plt.xlabel("iteration")
plt.ylabel("cost")
plt.legend(loc='upper right')
plt.title("Energy values (train)")
plt.show()

# 3. Display the values of the chosen regularization parameters

---
\
 - 각각의 lambda 값을 어떻게 설정하였는지 print하였습니다.


In [0]:
print(colored(over_lambda, 'red'))
print(colored(right_lambda, 'green'))
print(colored(under_lambda, 'blue'))

# 4. Plot the training accuracy

---
\
 - 각각의 lambda에 대해서 accuracy가 어떻게 변하는지 시각화했습니다.


In [0]:
'''
Visualize accuracy
'''
plt.figure(figsize=(10, 7))
plt.plot(over_accuracy_list, color='red', label='over-fitting')
plt.plot(right_accuracy_list, color='green', label='just-right')
plt.plot(under_accuracy_list, color='blue', label='under-fitting')
plt.xlabel("iteration")
plt.ylabel("accuracy")
plt.legend(loc='upper right')
plt.title("Accuracy")
plt.show()

# 5. Write down the final training accuracy

---
\
 - 각 laaccuracy_list에서 마지막 값을 출력했습니다. 



In [0]:
print("<Final training accuracy>")
print('-'*30)
over_acc='over-fitting accuracy : '+str(over_accuracy_list[-1]*100) + '%'
right_acc='just-right accuracy : '+str(right_accuracy_list[-1]*100) + '%'
under_acc='under-fitting accuracy : '+str(under_accuracy_list[-1]*100) + '%'
print(colored(over_acc, 'red'))
print(colored(right_acc, 'green'))
print(colored(under_acc, 'blue'))
print('-'*30)
print(max(over_accuracy_list))
print(max(right_accuracy_list))
print(max(under_accuracy_list))
print(under_theta)


In [0]:
'''
Visualize classifier
'''
print(over_grid_H)
print(under_grid_H)
fig = plt.figure(figsize=(10, 7))
plt.contour(grid_x, grid_y, over_grid_H, levels=[0.5], colors='red')
plt.contour(grid_x, grid_y, np.array(right_grid_H), levels=[0.5], colors='green')
plt.contour(grid_x, grid_y, np.array(under_grid_H), levels=[0.5], colors='blue')
plt.scatter(pointX0, pointY0, alpha=1, c='b', s=5)
plt.scatter(pointX1, pointY1, alpha=1, c='r', s=5)
plt.show()